In [ ]:
# Import necessary libraries
import cv2 as cv
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = Path('gdrive/My Drive/Dataset/Image/')

In [ ]:
# Define paths to the dataset
dry_path = path/'dry'
snowy_path = path/'snowy'
wet_path = path/'wet'

In [ ]:
# Initialize image size and batch size
image_size = 128
batch_size = 32

In [ ]:
# Load and preprocess the images
def load_images(path):
    images = []
    labels = []
    for subdir in ['dry', 'snowy', 'wet']:
        subpath = path / subdir
        for filename in os.listdir(subpath):
            img = cv.imread(os.path.join(subpath, filename))
            img = cv.resize(img, (image_size, image_size))
            img = img.astype('float32') / 255.0
            images.append(img)
            labels.append(subdir)
    return np.array(images), np.array(labels)

In [ ]:
# Load the dataset
X, y = load_images(path)

In [ ]:
# Convert labels to integers if they are not already one-hot encoded
if y.shape[1] != 3:  # Check if y is not already one-hot encoded
    label_map = {'dry': 0, 'snowy': 1, 'wet': 2}
    y_int = np.array([label_map[label] for label in y])  # Convert labels to integers
    y_encoded = tf.keras.utils.to_categorical(y_int)  # One-hot encode the labels
else:
    y_encoded = y  # y is already one-hot encoded, so no conversion is needed

# Split the dataset into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
# Create an ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

# Prepare the training dataset
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

In [ ]:
# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # 3 output classes: dry, snowy, wet
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_generator, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
25/25 [==============================] - 40s 1s/step - loss: 0.9345 - accuracy: 0.7599 - val_loss: 13.4076 - val_accuracy: 0.7203
Epoch 2/10
25/25 [==============================] - 30s 1s/step - loss: 0.6532 - accuracy: 0.7867 - val_loss: 13.2028 - val_accuracy: 0.7203
Epoch 3/10
25/25 [==============================] - 33s 1s/step - loss: 0.6403 - accuracy: 0.7867 - val_loss: 14.2199 - val_accuracy: 0.7203
Epoch 4/10
25/25 [==============================] - 30s 1s/step - loss: 0.6414 - accuracy: 0.7867 - val_loss: 13.5961 - val_accuracy: 0.7203
Epoch 5/10
25/25 [==============================] - 31s 1s/step - loss: 0.6407 - accuracy: 0.7867 - val_loss: 13.7847 - val_accuracy: 0.7203
Epoch 6/10
25/25 [==============================] - 30s 1s/step - loss: 0.6441 - accuracy: 0.7867 - val_loss: 14.4952 - val_accuracy: 0.7203
Epoch 7/10
25/25 [==============================] - 32s 1s/step - loss: 0.6415 - accuracy: 0.7867 - val_loss: 13.2717 - val_accuracy: 0.7203
Epoch 8/10
25

In [ ]:
# Save the trained model
model.save('road_surface_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load the model for inference
model = tf.keras.models.load_model('road_surface_model.h5')

In [ ]:
def classify_road_surface(image_path):
    img = cv.imread(str(image_path))
    img = cv.resize(img, (image_size, image_size))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    return np.argmax(prediction)

# Example usage:
image_path = path/'wet/wet00009.jpg'
road_condition = classify_road_surface(image_path)

1/1 [==============================] - 0s 31ms/step


In [ ]:
# Define class labels
class_labels = {0: 'Dry', 1: 'Snowy', 2: 'Wet'}

# Print the predicted road condition
print(f"The predicted road condition is: {class_labels[road_condition]}")

The predicted road condition is: Dry
